<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Einführung Machine Learning
### Sommersemester 2022
Prof. Dr. Heiner Giefers

# Fashion MNIST mit Neuronalen Netzen

In diesem Arbeitsblatt wollen wir uns erneut den Fashion MNIST Datensatz vornehmen, den wir schon aus dem 7. Arbeitsblatt kennen.
Dort haben wir das Problem mit Multinomialer Logistischer Regression gelöst.
Hier wollen wir ein Multi-Layer Perceptron einsetzen, also ein recht einfaches, mehrschichtiges (und voll-vermaschtes) Neuronale Netz.
Wir werden später sehen, dass sich eine andere Form von Neuronalen Netzen, die sogenannten Faltungsnetze (oder *Convolutional Neural Networks*, CNN) besser für solche Aufgaben eignen.

Zunächst importieren wir den Datensatz aus Tensorflow und passen die Daten an.

In [ ]:
import tensorflow as tf

#Datensatz aus Keras laden
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

#Pixelwerte nach [0,1] skalieren
X_train = X_train / 255.0
X_test = X_test / 255.0

# Mache aus den 2D Bildern 1D Vektoren
X_train = X_train.reshape(-1,28*28,1)[:,:,0]
X_test = X_test.reshape(-1,28*28,1)[:,:,0]

Der MLPClassifier aus **Scikit-learn**kann für binäre, Multi-Klassen und Multi-Label Klassifizierung verwendet werden. Die Aktivierungsfunktion der Ausgabeschicht wird basierend auf dem Typ von `y` ausgewählt:

- **Multi-Klassen**: Es sind verschiedene Klassen möglich, von denen eine wahr ist. `y` kann kann einen Wert annehmen (`y` ist ein Integer) $\rightarrow$ *Softmax*
- **Multi-Label oder binär**: Es sind verschiedene Klassen möglich, von denen mehrere wahr sein können (`y` ist ein Vektor von "Einzel-Wahrscheinlichkeiten") $\rightarrow$ *Sigmoidfunktion*
- **Regression**: `y` ist ein Wert aus $\mathbb{R}$ $\rightarrow$ *Identitätsfunktion* ($f(x)=x$)


Der MLPClassifier vervwendet als Trainings-Algorithmus *Stochastic Gradient Descent* (SGD), *Adam* oder *L-BFGS*.
SGD kennen wir bereits.
Adam ist eine Weiterentwicklung von SGD, bei der für einzelne Attribute die Lernrate entsprechend optimiert wird.
Dass kann dabei helfen, für jedes Merkmal die *passende* Schrittweite zu finden.

L-BFGS ist ebenfalls ein itteratives Lösungsverfahren und gehört zur Klasse der sogenannten *Quasi-Newton-Verfahren*.
Anders als SGD-basierte verfahren benötigt L-BFGS in jedem Iterationsschritt alle Daten, konvergiert aber in vielen Fällen trozdem recht schnell zum Minimum.

Da das Trainieren eines MLP Modells schon recht zeitaufwändig ist, verwenden wir in der folgenden Code-Zelle das Modul *Pickle*, um das Modell in eine Datei zu speichern.
Achten Sie darauf, die Datei zu löschen, wenn Sie das Modell erneut trainieren möchten.

In [ ]:
from sklearn.neural_network import MLPClassifier
import os, pickle
modelname = "model.pickle"

if os.path.isfile(modelname):
    clf = pickle.load(open(modelname, "rb"))
else:
    clf = MLPClassifier(solver='sgd', hidden_layer_sizes=(128,), random_state=1)
    clf.fit(X_train, y_train)
    pickle.dump(clf, open(modelname, "wb"))

In [ ]:
clf.score(X_test, y_test)

In [ ]:
# Zum Löschen der Modell-Datei folgende Zeite un-kommentieren:
# !rm model.pickle

## Fashion MNIST mit Keras

Nun Wollen wir statt Scikit-learn Tensorflow mit Keras verwenden, um ein MLP für Fashion MNIST zu trainieren.
Zuerst verwenden wir die **Sequential API**:

In [ ]:
from tensorflow import keras

#Datensatz aus Keras laden
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

#Pixelwerte nach [0,1] skalieren
X_train = X_train / 255.0
X_test = X_test / 255.0

#Modell definieren
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(128, activation=tf.nn.relu))
model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

#Modell erzeugen
model.compile('sgd','sparse_categorical_crossentropy',['accuracy'])

#Modell trainieren
model.fit(X_train, y_train, epochs=10)

#Trainiertes Modell auswerten
test_loss, test_acc = model.evaluate (X_test, y_test)
print('Test accuracy:', test_acc)

In [ ]:
model.summary()

Und nun die **Functional API**:

In [ ]:
#Datensatz aus Keras laden
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

#Pixelwerte nach [0,1] skalieren
X_train = X_train / 255.0
X_test = X_test / 255.0

#Funktionale abhängigkeiten
inputs = keras.Input(shape=(28, 28))
finputs = keras.layers.Flatten()(inputs)
l1 = keras.layers.Dense(128, activation=tf.nn.relu)(finputs)
outputs = keras.layers.Dense(10, activation=tf.nn.softmax)(l1)

#Modell definieren
model = keras.Model(inputs, outputs)

#Modell erzeugen
model.compile('sgd','sparse_categorical_crossentropy',['accuracy'])

#Modell trainieren
model.fit(X_train, y_train, epochs=5)

#Trainiertes Modell auswerten
test_loss, test_acc = model.evaluate (X_test, y_test)
print('Test accuracy:', test_acc)

### Referenzen

[1] Tensorflow.org, *Basic classification: Classify images of clothing*, tensorflow.org/tutorials/keras/classification, 2021
